DATA SOURCES:

    https://www.kaggle.com/whitefero/nba-player-salary-19902017

    https://data.world/the-pudding/winningest-cities-in-sports/workspace/file?filename=titles.csv



In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
csv_titles = "Resources/titles.csv"
titles_data_df = pd.read_csv(csv_titles)


csv_salaries = "Resources/salaries.csv"
salaries_data_df = pd.read_csv(csv_salaries)
titles_data_df

,index,year,level,sport,winner,winner_metro,runner_up,runner_up_metro,final_four3,final_four3_metro,final_four4,final_four4_metro
0,1,1870,college,Football (M),Princeton Tigers,"Princeton, NJ",NaN,NaN,NaN,NaN,NaN,NaN
1,2,1870,college,Football (M),Rutgers Scarlet Knights,"New Brunswick, NJ",NaN,NaN,NaN,NaN,NaN,NaN
2,3,1871,college,Football (M),Princeton Tigers,"Princeton, NJ",NaN,NaN,NaN,NaN,NaN,NaN
3,4,1873,college,Football (M),Princeton Tigers,"Princeton, NJ",NaN,NaN,NaN,NaN,NaN,NaN
4,5,1874,college,Football (M),Princeton Tigers,"Princeton, NJ",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
991,1001,2018,college,Basketball (W),Notre Dame Fighting Irish,"Notre Dame, IN",Mississippi State Bulldogs,"Starkville, MS",Louisville Cardinals,"Louisville, KY",Connecticut Huskies,"Hartford, CT"
992,1002,2018,pro,NHL,Washington Capitals,"Washington, DC Metro Area",Vegas Golden Knights,"Las Vegas, NV",Winnipeg Jets,"Winnipeg, MB",Tampa Bay Lightning,"Tampa Bay Area, FL"
993,1003,2018,pro,NBA,Golden State Warriors,"San Francisco Bay Area, CA",Cleveland Cavaliers,"Cleveland, OH",Boston Celtics,"Greater Boston, MA",Houston Rockets,"Houston, TX"
994,1004,2018,college,Baseball (M),Oregon State Beavers,"Corvallis, OR",Arkansas Razorbacks,"Northwest Arkansas, AR",Mississippi State Bulldogs,"Starkville, MS",Florida Gators,"Gainesville, FL"


In [3]:
salaries_data_df

,Register Value,Player Name,Salary in $,Season Start,Season End,Team,Full Team Name
0,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,354,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,727,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,1103,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,1497,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns
...,...,...,...,...,...,...,...
11832,6283,Zydrunas Ilgauskas,8740000,2005,2006,CLE,Cleveland Caveliers
11833,6658,Zydrunas Ilgauskas,9442697,2006,2007,CLE,Cleveland Caveliers
11834,7063,Zydrunas Ilgauskas,10142156,2007,2008,CLE,Cleveland Caveliers
11835,7485,Zydrunas Ilgauskas,10841615,2008,2009,CLE,Cleveland Caveliers


In [4]:
salaries_data_df = salaries_data_df.rename(columns={"Register Value": "Register_Value",
                                                    "Player Name": "Player_Name",
                                                    " Salary in $ ": "Salary_Dollars",
                                                    "Season Start": "Season_Start",
                                                    "Season End" :  "Season_End",
                                                    "Full Team Name" : "Full_Team_name"})
salaries_data_df

,Register_Value,Player_Name,Salary_Dollars,Season_Start,Season_End,Team,Full_Team_name
0,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,354,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,727,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,1103,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,1497,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns
...,...,...,...,...,...,...,...
11832,6283,Zydrunas Ilgauskas,8740000,2005,2006,CLE,Cleveland Caveliers
11833,6658,Zydrunas Ilgauskas,9442697,2006,2007,CLE,Cleveland Caveliers
11834,7063,Zydrunas Ilgauskas,10142156,2007,2008,CLE,Cleveland Caveliers
11835,7485,Zydrunas Ilgauskas,10841615,2008,2009,CLE,Cleveland Caveliers


In [5]:
salaries_data_df = salaries_data_df.replace({'Boston Celtic': 'Boston Celtics'})
salaries_data_df

,Register_Value,Player_Name,Salary_Dollars,Season_Start,Season_End,Team,Full_Team_name
0,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,354,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,727,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,1103,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,1497,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns
...,...,...,...,...,...,...,...
11832,6283,Zydrunas Ilgauskas,8740000,2005,2006,CLE,Cleveland Caveliers
11833,6658,Zydrunas Ilgauskas,9442697,2006,2007,CLE,Cleveland Caveliers
11834,7063,Zydrunas Ilgauskas,10142156,2007,2008,CLE,Cleveland Caveliers
11835,7485,Zydrunas Ilgauskas,10841615,2008,2009,CLE,Cleveland Caveliers


In [ ]:
#salaries_data_df.loc[salaries_data_df["Full_Team_name"] == "Boston Celtics"]
#salaries_data_df.head()

In [6]:
titles_data_pro = titles_data_df.loc[titles_data_df["level"]=="pro"]
titles_data_sport = titles_data_pro.loc[titles_data_pro["sport"] == "NBA"]
titles_data_sport.dropna()

,index,year,level,sport,winner,winner_metro,runner_up,runner_up_metro,final_four3,final_four3_metro,final_four4,final_four4_metro
236,243,1947,pro,NBA,Philadelphia Warriors,"Greater Philadelphia, PA",Chicago Stags,"Chicago, IL",New York Knicks,New York Metro Area,Washington Capitols,"Washington, DC Metro Area"
244,251,1948,pro,NBA,Baltimore Bullets,"Baltimore, MD",Philadelphia Warriors,"Greater Philadelphia, PA",Chicago Stags,"Chicago, IL",St. Louis Bombers,ERROR
252,259,1949,pro,NBA,Minneapolis Lakers,"Twin Cities, MN",Washington Capitols,"Washington, DC Metro Area",Rochester Royals,"Rochester, NY",New York Knicks,New York Metro Area
261,268,1950,pro,NBA,Minneapolis Lakers,"Twin Cities, MN",Syracuse Nationals,"Syracuse, NY",New York Knicks,New York Metro Area,Fort Wayne Pistons,"Fort Wayne, IN"
268,275,1951,pro,NBA,Rochester Royals,"Rochester, NY",New York Knicks,New York Metro Area,Minneapolis Lakers,"Twin Cities, MN",Syracuse Nationals,"Syracuse, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...
939,949,2014,pro,NBA,San Antonio Spurs,"San Antonio, TX",Miami Heat,"Greater Miami Area, FL",Indiana Pacers,"Indianapolis Metro Area, IN",Oklahoma City Thunder,"Oklahoma City, OK"
952,962,2015,pro,NBA,Golden State Warriors,"San Francisco Bay Area, CA",Cleveland Cavaliers,"Cleveland, OH",Atlanta Hawks,"Atlanta, GA",Houston Rockets,"Houston, TX"
966,976,2016,pro,NBA,Cleveland Cavaliers,"Cleveland, OH",Golden State Warriors,"San Francisco Bay Area, CA",Toronto Raptors,"Toronto, ON",Oklahoma City Thunder,"Oklahoma City, OK"
979,989,2017,pro,NBA,Golden State Warriors,"San Francisco Bay Area, CA",Cleveland Cavaliers,"Cleveland, OH",San Antonio Spurs,"San Antonio, TX",Boston Celtics,"Greater Boston, MA"


In [7]:
#Create connection to postgres

connection_string = "postgres:postgres@localhost:5432/NBA_db"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# add titles table
titles_data_sport.to_sql(name='titles', con=engine, if_exists='append', index=False)

In [9]:
#add salaries table
salaries_data_df.to_sql(name='salaries', con=engine, if_exists='append', index=False)

In [10]:
#confirm titles table
pd.read_sql_query('select * from titles', con=engine)

,index,year,level,sport,winner,winner_metro,runner_up,runner_up_metro,final_four3,final_four3_metro,final_four4,final_four4_metro
0,243,1947,pro,NBA,Philadelphia Warriors,"Greater Philadelphia, PA",Chicago Stags,"Chicago, IL",New York Knicks,New York Metro Area,Washington Capitols,"Washington, DC Metro Area"
1,251,1948,pro,NBA,Baltimore Bullets,"Baltimore, MD",Philadelphia Warriors,"Greater Philadelphia, PA",Chicago Stags,"Chicago, IL",St. Louis Bombers,ERROR
2,259,1949,pro,NBA,Minneapolis Lakers,"Twin Cities, MN",Washington Capitols,"Washington, DC Metro Area",Rochester Royals,"Rochester, NY",New York Knicks,New York Metro Area
3,268,1950,pro,NBA,Minneapolis Lakers,"Twin Cities, MN",Syracuse Nationals,"Syracuse, NY",New York Knicks,New York Metro Area,Fort Wayne Pistons,"Fort Wayne, IN"
4,275,1951,pro,NBA,Rochester Royals,"Rochester, NY",New York Knicks,New York Metro Area,Minneapolis Lakers,"Twin Cities, MN",Syracuse Nationals,"Syracuse, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...
67,949,2014,pro,NBA,San Antonio Spurs,"San Antonio, TX",Miami Heat,"Greater Miami Area, FL",Indiana Pacers,"Indianapolis Metro Area, IN",Oklahoma City Thunder,"Oklahoma City, OK"
68,962,2015,pro,NBA,Golden State Warriors,"San Francisco Bay Area, CA",Cleveland Cavaliers,"Cleveland, OH",Atlanta Hawks,"Atlanta, GA",Houston Rockets,"Houston, TX"
69,976,2016,pro,NBA,Cleveland Cavaliers,"Cleveland, OH",Golden State Warriors,"San Francisco Bay Area, CA",Toronto Raptors,"Toronto, ON",Oklahoma City Thunder,"Oklahoma City, OK"
70,989,2017,pro,NBA,Golden State Warriors,"San Francisco Bay Area, CA",Cleveland Cavaliers,"Cleveland, OH",San Antonio Spurs,"San Antonio, TX",Boston Celtics,"Greater Boston, MA"


In [11]:
#confirm salaries table
pd.read_sql_query('select * from salaries', con=engine)


,Register_Value,Player_Name,Salary_Dollars,Season_Start,Season_End,Team,Full_Team_name
0,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,354,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,727,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,1103,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,1497,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns
...,...,...,...,...,...,...,...
11832,6283,Zydrunas Ilgauskas,8740000,2005,2006,CLE,Cleveland Caveliers
11833,6658,Zydrunas Ilgauskas,9442697,2006,2007,CLE,Cleveland Caveliers
11834,7063,Zydrunas Ilgauskas,10142156,2007,2008,CLE,Cleveland Caveliers
11835,7485,Zydrunas Ilgauskas,10841615,2008,2009,CLE,Cleveland Caveliers


In [12]:
final_df = pd.read_sql_query('select * From salaries Join titles\
                            On salaries."Full_Team_name" = titles.winner', con=engine)
final_df

,Register_Value,Player_Name,Salary_Dollars,Season_Start,Season_End,Team,Full_Team_name,index,year,level,sport,winner,winner_metro,runner_up,runner_up_metro,final_four3,final_four3_metro,final_four4,final_four4_metro
0,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers,897,2010,pro,NBA,Los Angeles Lakers,"Greater Los Angeles, CA",Boston Celtics,"Greater Boston, MA",Orlando Magic,"Greater Orlando, FL",Phoenix Suns,"Metro Phoenix, AZ"
1,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers,885,2009,pro,NBA,Los Angeles Lakers,"Greater Los Angeles, CA",Orlando Magic,"Greater Orlando, FL",Cleveland Cavaliers,"Cleveland, OH",Denver Nuggets,"Denver, CO"
2,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers,792,2002,pro,NBA,Los Angeles Lakers,"Greater Los Angeles, CA",New Jersey Nets,New York Metro Area,Boston Celtics,"Greater Boston, MA",Sacramento Kings,"Sacramento, CA"
3,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers,780,2001,pro,NBA,Los Angeles Lakers,"Greater Los Angeles, CA",Philadelphia 76ers,"Greater Philadelphia, PA",Milwaukee Bucks,"Milwaukee, WI",San Antonio Spurs,"San Antonio, TX"
4,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers,766,2000,pro,NBA,Los Angeles Lakers,"Greater Los Angeles, CA",Indiana Pacers,"Indianapolis Metro Area, IN",New York Knicks,New York Metro Area,Portland Trail Blazers,"Portland Metro Area, OR"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23281,10716,Zoran Dragic,2050397,2015,2016,BOS,Boston Celtics,368,1962,pro,NBA,Boston Celtics,"Greater Boston, MA",Los Angeles Lakers,"Greater Los Angeles, CA",Detroit Pistons,"Metro Detroit, MI",Philadelphia 76ers,"Greater Philadelphia, PA"
23282,10716,Zoran Dragic,2050397,2015,2016,BOS,Boston Celtics,359,1961,pro,NBA,Boston Celtics,"Greater Boston, MA",St. Louis Hawks,"St. Louis, MO",Los Angeles Lakers,"Greater Los Angeles, CA",Syracuse Nationals,"Syracuse, NY"
23283,10716,Zoran Dragic,2050397,2015,2016,BOS,Boston Celtics,350,1960,pro,NBA,Boston Celtics,"Greater Boston, MA",St. Louis Hawks,"St. Louis, MO",Minneapolis Lakers,"Twin Cities, MN",Philadelphia Warriors,"Greater Philadelphia, PA"
23284,10716,Zoran Dragic,2050397,2015,2016,BOS,Boston Celtics,342,1959,pro,NBA,Boston Celtics,"Greater Boston, MA",Minneapolis Lakers,"Twin Cities, MN",St. Louis Hawks,"St. Louis, MO",Syracuse Nationals,"Syracuse, NY"


In [13]:
final_df.count()

Register_Value       23286
Player_Name          23286
Salary_Dollars       23286
Season_Start         23286
Season_End           23286
Team                 23286
Full_Team_name       23286
index                23286
year                 23286
level                23286
sport                23286
winner               23286
winner_metro         23286
runner_up            23286
runner_up_metro      23286
final_four3          23286
final_four3_metro    23286
final_four4          23286
final_four4_metro    23286
dtype: int64

In [16]:
team_year = pd.read_sql_query('select SUM (salaries."Salary_Dollars") AS Team_Total,\
                              salaries."Full_Team_name" from salaries INNER JOIN titles \
                              On salaries."Full_Team_name" = titles.winner Group by "Full_Team_name";', con=engine)
team_year

,team_total,Full_Team_name
0,"2,461,037,512.00",Philadelphia 76ers
1,"7,603,841,346.00",Chicago Bulls
2,"3,887,116,653.00",Miami Heat
3,"1,430,460,821.00",Milwaukee Bucks
4,"5,599,644,824.00",Golden State Warriors
5,"18,367,725,739.00",Los Angeles Lakers
6,"22,328,329,261.00",Boston Celtics
7,"2,518,207,752.00",Houston Rockets
8,"1,493,116,122.00",Dallas Mavericks
9,"3,442,728,984.00",New York Knicks


In [15]:
team_year = pd.options.display.float_format = '{:,.2f}'.format
team_year

<function str.format>